# vLLM efficient inference

In [1]:
from importlib.metadata import version

In [2]:
version('vllm')

'0.11.0'

## vLLM 0.11.0 documentation

https://docs.vllm.ai/en/stable

### Python API

Quick start

https://docs.vllm.ai/en/latest/getting_started/quickstart/#offline-batched-inference

Examples

https://docs.vllm.ai/en/latest/examples/offline_inference/async_llm_streaming/

https://docs.vllm.ai/en/latest/examples/offline_inference/batch_llm_inference/

User guide

https://docs.vllm.ai/en/latest/serving/offline_inference/

https://docs.vllm.ai/en/latest/models/generative_models/

https://docs.vllm.ai/en/latest/models/pooling_models/

API reference

https://docs.vllm.ai/en/latest/api/

https://docs.vllm.ai/en/latest/api/vllm/#vllm.LLM

Config arguments you can pass

- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.ModelConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.CacheConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.LoadConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.ParallelConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.SchedulerConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.DeviceConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.SpeculativeConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.LoRAConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.MultiModalConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.PoolerConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.StructuredOutputsConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.ObservabilityConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.KVTransferConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.CompilationConfig
- https://docs.vllm.ai/en/latest/api/vllm/config/#vllm.config.VllmConfig

Supported models

https://docs.vllm.ai/en/latest/models/supported_models/

https://github.com/vllm-project/vllm/tree/main/vllm/model_executor/models

### OpenAI-Compatible RESTful API server

Quick start

https://docs.vllm.ai/en/latest/getting_started/quickstart/#openai-compatible-server

Examples

https://docs.vllm.ai/en/latest/examples/online_serving/openai_chat_completion_client/

User guide

https://docs.vllm.ai/en/latest/serving/openai_compatible_server/

Configuration

https://docs.vllm.ai/en/latest/configuration/

Syntax reference

https://docs.vllm.ai/en/latest/cli/

https://docs.vllm.ai/en/latest/cli/serve/

https://docs.vllm.ai/en/latest/configuration/serve_args/

## Streaming a response in the notebook

In [2]:
import asyncio

from vllm import SamplingParams
from vllm.engine.arg_utils import AsyncEngineArgs
from vllm.sampling_params import RequestOutputKind
from vllm.v1.engine.async_llm import AsyncLLM
 
def start_vllm_engine(model: str, **kwargs):
    engine_args = AsyncEngineArgs(
        model=model,
        enforce_eager=True,  # Faster startup for examples
        **kwargs
    )
    engine = AsyncLLM.from_engine_args(engine_args)
    return engine

def stop_vllm_engine(engine: AsyncLLM):
    engine.shutdown()

async def stream_vllm_response(engine: AsyncLLM, prompt: str, request_id = "default") -> None:
    sampling_params = SamplingParams(
        max_tokens=4096,
        temperature=0.8,
        top_p=0.95,
        seed=42,  # For reproducible results
        output_kind=RequestOutputKind.DELTA,  # Get only new tokens each iteration
    )

    try:
        # Stream tokens from AsyncLLM
        async for output in engine.generate(
            request_id=request_id, prompt=prompt, sampling_params=sampling_params
        ):            
            # Process each completion in the output
            for completion in output.outputs:
                # In DELTA mode, we get only new tokens generated since last iteration
                new_text = completion.text
                if new_text:
                    print(new_text, end="", flush=True)

            # Check if generation is finished
            if output.finished:
                print("\n✅ Generation complete!")
                break

    except Exception as e:
        print(f"\n❌ Error during streaming: {e}")
        raise

In [3]:
engine = start_vllm_engine(model="Qwen/Qwen3-4B-Thinking-2507-FP8", max_model_len=32768)

INFO 11-13 22:31:54 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 11-13 22:31:54 [model.py:1510] Using max model len 32768


2025-11-13 22:31:55,892	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 11-13 22:31:55 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 11-13 22:31:55 [__init__.py:381] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=27594) INFO 11-13 22:31:56 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=27594) INFO 11-13 22:31:56 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen3-4B-Thinking-2507-FP8', speculative_config=None, tokenizer='Qwen/Qwen3-4B-Thinking-2507-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, dis

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.50s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:03<00:00,  3.50s/it]
(EngineCore_DP0 pid=27594) 


(EngineCore_DP0 pid=27594) INFO 11-13 22:32:02 [default_loader.py:267] Loading weights took 3.55 seconds
(EngineCore_DP0 pid=27594) INFO 11-13 22:32:02 [gpu_model_runner.py:2653] Model loading took 4.2299 GiB and 4.377461 seconds
(EngineCore_DP0 pid=27594) WARNING 11-13 22:32:02 [fp8_utils.py:576] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /home/python/archive-v0/anz9liZrnnyqrcULZj6es/vllm/model_executor/layers/quantization/utils/configs/N=6144,K=2560,device_name=NVIDIA_GeForce_RTX_4090,dtype=fp8_w8a8,block_shape=[128,128].json
(EngineCore_DP0 pid=27594) WARNING 11-13 22:32:03 [fp8_utils.py:576] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /home/python/archive-v0/anz9liZrnnyqrcULZj6es/vllm/model_executor/layers/quantization/utils/configs/N=2560,K=4096,device_name=NVIDIA_GeForce_RTX_4090,dtype=fp8_w8a8,block_shape=[128,128].json
(EngineCore_DP0 pid=27594) WARNING 11-13 2

In [4]:
await stream_vllm_response(engine, "Explain how transformers use attention to process language.", "1")

 In your explanation, include at most two sentences about the relationship between attention and language processing.

Okay, the user wants me to explain how transformers use attention for language processing, with a specific constraint: I can only include two sentences about the relationship between attention and language processing. 

Hmm, this seems like someone studying NLP or machine learning who needs a concise yet precise explanation. They're probably preparing for an exam or writing a report where brevity matters. I should avoid jargon overload while staying technically accurate.

First, I recall that transformers' core innovation is self-attention. Each token gets a vector that weighs all other tokens' relevance through attention scores. The key is that this allows modeling long-range dependencies without RNNs' sequential limitations. 

For the two-sentence requirement, I'll focus on: (1) how attention computes weighted relationships between tokens, and (2) why this matters fo

In [5]:
stop_vllm_engine(engine)